In [1]:
import os
import cv2
import torch
import roboflow
import base64

import supervision as sv
import numpy as np

from roboflow import Roboflow
from fastsam import FastSAM, FastSAMPrompt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from matplotlib.backends.backend_agg import FigureCanvasAgg

print("DONE")

DONE


In [10]:
#!wget -P {HOME}/weights -q https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt
print("DONE")
!python --version

DONE
Python 3.10.14


In [5]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"DEVICE = {DEVICE}")
fast_sam = FastSAM("FastSAM.pt")

DEVICE = cpu


In [2]:
# SELECT FILE TO USE, AS WELL AS NAME FOR NEW FILE TO BE SAVED

print("Enter file to use with extension")
file_n = input()
print("Enter filename to save as without extension")
file2_n = input()
print(file_n, file2_n)
print("DONE")

Enter file to use with extension


 people.jpeg


Enter filename to save as without extension


 people2


people.jpeg people2
DONE


In [3]:
img_r = "images/"
IMAGE_PATH = f"{img_r}{file_n}"
output_p = f"{'./output'}{'/'}{file2_n}{'.jpg'}"
print(IMAGE_PATH, output_p)

images/people.jpeg ./output/people2.jpg


In [ ]:
#os.makedirs(f"./output", exist_ok=True)

In [6]:
results = fast_sam(
    source=IMAGE_PATH,
    device=DEVICE,
    retina_masks=True,
    imgsz=1024,
    conf=0.4,
    iou=0.9)
prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
masks = prompt_process.everything_prompt()
prompt_process.plot(annotations=masks, output_path=f"{'./output/'}{file2_n}{'.jpg'}")
print("done with this")


image 1/1 /Users/josephmakokha/FastSAM/images/people.jpeg: 768x1024 14 objects, 2368.8ms
Speed: 4.1ms preprocess, 2368.8ms inference, 14.5ms postprocess per image at shape (1, 3, 1024, 1024)
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..255.0].
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [0.0..255.0].


done with this


In [7]:
# Convert masks to boolean (True/False)
def masks_to_bool(masks):
    if type(masks) == np.ndarray:
        return masks.astype(bool)
    return masks.cpu().numpy().astype(bool)
print("done with this too")

done with this too


In [8]:
def annotate_image(image_path: str, masks: np.ndarray) -> np.ndarray:
    image = cv2.imread(image_path)

    xyxy = sv.mask_to_xyxy(masks=masks)
    detections = sv.Detections(xyxy=xyxy, mask=masks)

    mask_annotator = sv.MaskAnnotator(color_lookup = sv.ColorLookup.INDEX)
    return mask_annotator.annotate(scene=image.copy(), detections=detections)
print("done")

done


In [9]:
masks = masks_to_bool(masks)
annotated_image=annotate_image(image_path=IMAGE_PATH, masks=masks)
sv.plot_image(image=annotated_image, size=(8, 8))